In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import re

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://lsbrj.web.geniussports.com/competitions/?WHurl=%2Fcompetition%2F35583%2Fteams'
driver.get(website)

driver.maximize_window()

camp_links = []
equipes = []
players = []
names_players = []

# Espera pelo elemento pai que contém os anos
elemento_pai = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="yearChooser"]')))
option_anos = elemento_pai.find_elements(By.XPATH, './/option')

# Filtrar apenas o ano de 2024
for ano in option_anos:
    if ano.text.strip() == '2024':
        ano.click()
        break

# Espera pelo container dos campeonatos
container = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="comp-selector"]')))
products = container.find_elements(By.XPATH, './/select[@id="competitionChooser"]')
links = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/option')))

# Coletar os links dos campeonatos
for link in links:
    value = link.get_attribute("value")
    if value and value.startswith("http"):
        camp_links.append(value)

# Remover duplicatas e valores vazios, mantendo apenas os campeonatos de 2024
cl_filtered = list(set(camp_links))
print(cl_filtered)
print(len(cl_filtered))

# Coletar os links das equipes
for link in cl_filtered:
    try:
        driver.get(link)

        container_2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="teams-list-page page showlogos"]')))
        div_elements = container_2.find_elements(By.XPATH, './/div[contains(@class, "team-link")]')

        for div_element in div_elements:
            a_element = div_element.find_element(By.XPATH, './/a')
            href = a_element.get_attribute('href')
            if href and href.startswith("http"):
                equipes.append(href)
    
    except TimeoutException:
        print(f"TimeoutException occurred for link: {link}")

equipes = list(set(equipes))
print(equipes)
print(len(equipes))

# Coletar os links dos jogadores
for link_2 in equipes:
    try:
        driver.get(link_2)

        container_3 = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//table[@class="tableClass team-players-table team-stats footable footable-done footable-1 breakpoint-lg"]')))
        td_elements = container_3.find_elements(By.XPATH, './/td[@class="footable-first-visible"]')

        for td_element in td_elements:
            try:
                a_element_2 = td_element.find_element(By.XPATH, './/a')
                href = a_element_2.get_attribute('href')
                if href and href.startswith("http"):
                    players.append(href)
            except Exception as e:
                print(f"Error while extracting player link: {e}")
    
    except TimeoutException:
        print(f"TimeoutException occurred for link: {link_2}")

players = list(set(players))
print(players)
print(len(players))

# Coletar os nomes dos jogadores
for link_3 in players:
    try:
        driver.get(link_3)

        container_4 = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//div[@class="person-name fake-cell"]')))
        h1_elements = container_4.find_elements(By.XPATH, './/h1[@class="person-title"]')
    
        for h1_element in h1_elements:
            player_name = h1_element.text
            names_players.append(player_name)
    except TimeoutException:
        print(f"TimeoutException occurred for link: {link_3}")

names_players = list(set(names_players))
print(names_players)
print(len(names_players))

df_list = pd.DataFrame({'Membro': names_players})


['https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37972%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37974%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F35583%2Fteams&_lng=pt', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37876%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37971%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37970%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37973%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37874%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37875%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37914%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F37969%2Fteams', 'https://lsbrj.web.geniussports.com/?p=9&WHurl=%2Fcompetition%2F35583%2Fteams&_lng=en', 'https://lsbrj.web.geni

In [3]:
df_list.to_csv('LSB_Basquete_Atletas_V2_2024.csv', index=False)